In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 1. Представление и предобработка текстовых данных

1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел


In [ ]:
def preprocess_text(text: str) -> str:
    # Приведение строки к нижнему регистру
    text = text.lower()

    # Замена всех символов, кроме a-z, A-Z на пробел
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Токенизация
    tokens = word_tokenize(text)

    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Удаление стоп-слов
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]

    # Объединение токенов в строку
    processed_text = ' '.join(filtered_tokens)

    return processed_text

# Пример использования
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'
processed_text = preprocess_text(text)
print("Original Text: ", text)
print("Processed Text: ", processed_text)

Original Text:  Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++
Processed Text:  select preference run install command stable represents currently tested supported version pytorch note libtorch available c


1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [ ]:
# Пример текста
text = 'Select your preferences and run the install command.'

# Токенизация предложения
tokens = word_tokenize(text)

# Создание словаря уникальных слов
word_to_index = {word: i for i, word in enumerate(set(tokens))}

# Представление предложения в виде тензора бинарного кодирования
sentence_t = torch.zeros(len(word_to_index))

for token in tokens:
    if token in word_to_index:
        sentence_t[word_to_index[token]] = 1

print("Original Sentence: ", text)
print("Binary Tensor: ", sentence_t)


Original Sentence:  Select your preferences and run the install command.
Binary Tensor:  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.])


## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [ ]:
import pandas as pd

# Считывание файла CSV в DataFrame
df = pd.read_csv("/content/drive/MyDrive/datasets/surnames.csv")

In [ ]:
df

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian
...,...,...
10975,Quraishi,Arabic
10976,Innalls,English
10977,Król,Polish
10978,Purvis,English


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Инициализация LabelEncoder
label_encoder = LabelEncoder()

# Кодирование столбца 'nationality'
df['nationality_encoded'] = label_encoder.fit_transform(df['nationality'])

df

,surname,nationality,nationality_encoded
0,Woodford,English,4
1,Coté,French,5
2,Kore,English,4
3,Koury,Arabic,0
4,Lebzak,Russian,14
...,...,...,...
10975,Quraishi,Arabic,0
10976,Innalls,English,4
10977,Król,Polish,12
10978,Purvis,English,4


In [ ]:
from sklearn.model_selection import train_test_split

# Разделение на признаки (X) и целевую переменную (y)
X = df['surname']
y = df['nationality_encoded']

# Разделение на обучающую и тестовую выборку (80% - обучающая, 20% - тестовая)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Вывод размеров обучающей и тестовой выборок
print("Размер обучающей выборки:", X_train.shape[0])
print("Размер тестовой выборки:", X_test.shape[0])

Размер обучающей выборки: 8784
Размер тестовой выборки: 2196


In [ ]:
class Vocab:
    def __init__(self, data):
        self.idx_to_token, self.token_to_idx = self.build_vocab(data)
        self.vocab_len = len(self.idx_to_token)

    def build_vocab(self, data):
        unique_tokens = list(set(''.join(data)))
        idx_to_token = {i: token for i, token in enumerate(unique_tokens)}
        token_to_idx = {token: i for i, token in enumerate(unique_tokens)}
        return idx_to_token, token_to_idx

# Пример использования
data = df['surname']
vocab = Vocab(data)

print("Index to Token:", vocab.idx_to_token)
print("Token to Index:", vocab.token_to_idx)
print("Vocabulary Length:", vocab.vocab_len)


Index to Token: {0: 'c', 1: 'X', 2: 't', 3: ':', 4: 'p', 5: 'M', 6: 'x', 7: 'ß', 8: 'F', 9: 'i', 10: 'J', 11: 'è', 12: 'z', 13: 'G', 14: 'ù', 15: 'I', 16: 'Á', 17: 'r', 18: 'T', 19: 'R', 20: 'v', 21: 'P', 22: 'S', 23: 'Z', 24: 'N', 25: 'm', 26: '/', 27: 'U', 28: 'h', 29: 'g', 30: 'ł', 31: 'q', 32: 'w', 33: 'à', 34: 'ò', 35: 'l', 36: 'ã', 37: 'K', 38: 'ä', 39: 'O', 40: 'A', 41: 'f', 42: 'o', 43: 'ê', 44: 'ą', 45: 'ñ', 46: 'H', 47: 'ì', 48: 'ü', 49: 'a', 50: '1', 51: 'D', 52: 'ż', 53: 'n', 54: 'e', 55: 'B', 56: 'é', 57: 'Ż', 58: '-', 59: 'Q', 60: 'Y', 61: 'L', 62: 'á', 63: 'í', 64: 'õ', 65: 'C', 66: 'ń', 67: 'd', 68: 'ö', 69: 's', 70: "'", 71: 'ç', 72: 'ó', 73: 'j', 74: 'Ś', 75: 'b', 76: 'É', 77: 'V', 78: 'k', 79: 'W', 80: 'y', 81: 'ú', 82: 'u', 83: 'E'}
Token to Index: {'c': 0, 'X': 1, 't': 2, ':': 3, 'p': 4, 'M': 5, 'x': 6, 'ß': 7, 'F': 8, 'i': 9, 'J': 10, 'è': 11, 'z': 12, 'G': 13, 'ù': 14, 'I': 15, 'Á': 16, 'r': 17, 'T': 18, 'R': 19, 'v': 20, 'P': 21, 'S': 22, 'Z': 23, 'N': 24, 'm': 

In [ ]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, surname):
        vectorized_surname = torch.zeros(self.vocab.vocab_len)
        for char in surname:
            if char in self.vocab.token_to_idx:
                vectorized_surname[self.vocab.token_to_idx[char]] = 1
        return vectorized_surname

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        surname = self.X.iloc[idx]
        vectorized_surname = self.vectorize(surname)
        label = torch.tensor(self.y.iloc[idx])
        return vectorized_surname, label

# Пример использования
# (Предполагается, что у вас уже есть объект vocab)
dataset = SurnamesDataset(X_train, y_train, vocab)

# Пример доступа к данным
sample = dataset[0]
print("Vectorized Surname:", sample[0])
print("Label:", sample[1])

Vectorized Surname: tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
Label: tensor(4)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Пример простой модели
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

# Параметры
input_size = len(vocab.idx_to_token)  # Размер входного вектора (длина словаря)
output_size = len(set(y_train))  # Количество классов (уникальных национальностей)
learning_rate = 0.003
batch_size = 64
epochs = 20

# Создание объектов DataLoader для обучающей и тестовой выборок
train_dataset = SurnamesDataset(X_train, y_train, vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = SurnamesDataset(X_test, y_test, vocab)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Инициализация модели, функции потерь и оптимизатора
model = SimpleClassifier(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    total_loss = 0
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')

# Оценка модели на тестовой выборке
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on test data: {accuracy * 100:.2f}%')


Epoch 1/20, Loss: 2.780480149863423
Epoch 2/20, Loss: 2.6240784379019253
Epoch 3/20, Loss: 2.567544180413951
Epoch 4/20, Loss: 2.54147660213968
Epoch 5/20, Loss: 2.5267595432806704
Epoch 6/20, Loss: 2.51582061726114
Epoch 7/20, Loss: 2.5043855646382207
Epoch 8/20, Loss: 2.495388715163521
Epoch 9/20, Loss: 2.4881544216819433
Epoch 10/20, Loss: 2.4833891858225283
Epoch 11/20, Loss: 2.478823210882104
Epoch 12/20, Loss: 2.47559542586838
Epoch 13/20, Loss: 2.4722333887349004
Epoch 14/20, Loss: 2.467300057411194
Epoch 15/20, Loss: 2.4643397590388423
Epoch 16/20, Loss: 2.4616142338600713
Epoch 17/20, Loss: 2.4588810585547183
Epoch 18/20, Loss: 2.4573119066763613
Epoch 19/20, Loss: 2.456542217213175
Epoch 20/20, Loss: 2.4532931144686714
Accuracy on test data: 53.87%


In [ ]:
# Измерение точности на тестовой выборке
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on test data: {accuracy * 100:.2f}%')

# Проверка работоспособности модели на нескольких фамилиях студентов
student_surnames = ["Farkhutdinov","Królik"]

for surname in student_surnames:
    vectorized_surname = test_dataset.vectorize(surname)
    input_tensor = vectorized_surname.clone().detach().float().unsqueeze(0)
    output = model(input_tensor)

    # Получение индексов трех наиболее вероятных предсказаний
    _, top_indices = torch.topk(output, 3)

    # Преобразование индексов в национальности с использованием обратного словаря
    predicted_nationalities = [label_encoder.classes_[idx] for idx in top_indices.numpy()[0]]

    print(f"Surname: {surname}, Predictions: {predicted_nationalities}")


Accuracy on test data: 53.87%
Surname: Farkhutdinov, Predictions: ['Russian', 'English', 'Japanese']
Surname: Królik, Predictions: ['Russian', 'English', 'Arabic']


## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


In [ ]:
file_path = '/content/drive/MyDrive/datasets/yelp/raw_train.csv'
df = pd.read_csv(file_path, header=None, names=['rating', 'review'])

# Оставить только 10% строк
fraction = 0.1
df_sampled = df.sample(frac=fraction, random_state=42)
df_sampled

,rating,review
34566,2,This place is one of my favorite comic shops. ...
223092,1,The wait time for an appointment is ridiculous...
110270,1,I did not like this hotel at all. It's very ol...
365013,2,Mill Avenue has a serious issue with parking. ...
311625,2,Favorite sushi place in NV! Price is reasonab...
...,...,...
213529,2,It's time for an update of CVS -\n\nIf you are...
241422,1,"I was in the area doing some shopping, and bot..."
141719,1,I did an all Venetian restaurant tour on my mo...
39372,2,"Great atmosphere, good food, profesional servi..."


In [ ]:
df['rating'].value_counts()

1    280000
2    280000
Name: rating, dtype: int64

In [ ]:
df_sampled['review'] = df_sampled['review'].apply(preprocess_text)
label_encoder = LabelEncoder()
df_sampled['rating'] = label_encoder.fit_transform(df_sampled['rating'])

In [ ]:
df_sampled

,rating,review
34566,1,place one favorite comic shop actually live cl...
223092,0,wait time appointment ridiculous waiting hour ...
110270,0,like hotel old comfort nthe good thing wa chea...
365013,1,mill avenue ha serious issue parking fan vario...
311625,1,favorite sushi place nv price reasonable food ...
...,...,...
213529,1,time update cv n nif using extra care card uti...
241422,0,wa area shopping man decided wa time eat unfor...
141719,0,venetian restaurant tour recent vega trip dinn...
39372,1,great atmosphere good food profesional service...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_sampled['review'], df_sampled['rating'], test_size=0.2, random_state=42)

# Вывести информацию о размере обучающей и тестовой выборок
print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")

Размер обучающей выборки: 44800
Размер тестовой выборки: 11200


In [ ]:
class Vocab:
    def __init__(self, data):
        self.idx_to_token, self.token_to_idx = self.build_vocab(data)
        self.vocab_len = len(self.idx_to_token)

    def build_vocab(self, data):
        unique_tokens = list(set(' '.join(data).split()))  # Разбиваем текст на слова
        idx_to_token = {i: token for i, token in enumerate(unique_tokens)}
        token_to_idx = {token: i for i, token in enumerate(unique_tokens)}
        return idx_to_token, token_to_idx

# Пример использования
data = df_sampled['review']
vocab = Vocab(data)

print("Index to Token:", vocab.idx_to_token)
print("Token to Index:", vocab.token_to_idx)
print("Vocabulary Length:", vocab.vocab_len)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import torch
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, X, y, vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, review):
        # Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)
        vectorized_review = torch.zeros(len(self.vocab.idx_to_token))

        # Применяем предобработку текста
        processed_review = preprocess_text(review)

        # Бинарное кодирование слов в предложении
        for word in processed_review.split():
            if word in self.vocab.token_to_idx:
                vectorized_review[self.vocab.token_to_idx[word]] = 1

        return vectorized_review

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        review = self.X.iloc[idx]
        vectorized_review = self.vectorize(review)
        label = torch.tensor(self.y.iloc[idx])

        return vectorized_review, label


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Пример простой модели
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

# Параметры
input_size = len(vocab.idx_to_token)
output_size = len(set(y_train))  # Количество уникальных рейтингов
learning_rate = 0.001
batch_size = 64
epochs = 2

# Создание объектов DataLoader для обучающей и тестовой выборок
train_dataset = ReviewDataset(X_train, y_train, vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Инициализация модели, функции потерь и оптимизатора
model = SimpleClassifier(input_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    total_loss = 0
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')


Epoch 1/2, Loss: 0.5063796095762934
Epoch 2/2, Loss: 0.42660544953175955


In [ ]:
# Оценка модели на тестовой выборке
correct = 0
total = 0

test_dataset = ReviewDataset(X_test, y_test, vocab)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on test data: {accuracy * 100:.2f}%')

# Проверка работы модели на явно позитивном отзыве
positive_review = "This product is amazing! I love it."
vectorized_positive_review = test_dataset.vectorize(positive_review)
vectorized_positive_review = vectorized_positive_review.unsqueeze(0)  # Добавляем размерность батча
predicted_class_positive = torch.argmax(model(vectorized_positive_review.float()))

print(f'Predicted class for positive review: {predicted_class_positive.item()}')

# Проверка работы модели на явно негативном отзыве
negative_review = "This is terrible. I regret buying it."
vectorized_negative_review = test_dataset.vectorize(negative_review)
vectorized_negative_review = vectorized_negative_review.unsqueeze(0)  # Добавляем размерность батча
predicted_class_negative = torch.argmax(model(vectorized_negative_review.float()))

print(f'Predicted class for negative review: {predicted_class_negative.item()}')


Accuracy on test data: 91.35%
Predicted class for positive review: 1
Predicted class for negative review: 0
